In [1]:
!pip install geocoder
!pip install folium

In [103]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


Get the data from github

In [174]:
url = 'https://raw.githubusercontent.com/AxelllD/Coursera_Capstone/main/Shanghai%20Postal%20Code%20data.csv'
df = pd.read_csv(url, index_col=0)
df.head(5)

,Division,Area (km2)[105],Total population 2017[105],Seat,Postal code,Latitude,Longitude
Division code[104],,,,,,,
310000.0,Shanghai,6340.50,24183300.0,Huangpu,200000,NaN,NaN
310101.0,Huangpu,20.46,654800.0,Waitan Subdistrict,200001,31.23,121.48
NaN,Luwan,NaN,NaN,NaN,200020,31.22,121.47
NaN,Zhabei,NaN,NaN,NaN,200070,31.25,121.45
310104.0,Xuhui,54.76,1088300.0,Xujiahui Subdistrict,200030,31.18,121.43


Drop irrelevant columns such as area size, population and seat

In [175]:
df.drop([r'Area (km2)[105]', r'Total population 2017[105]', 'Seat'], axis=1, inplace=True)
df

,Division,Postal code,Latitude,Longitude
Division code[104],,,,
310000.0,Shanghai,200000,NaN,NaN
310101.0,Huangpu,200001,31.23,121.48
NaN,Luwan,200020,31.22,121.47
NaN,Zhabei,200070,31.25,121.45
310104.0,Xuhui,200030,31.18,121.43
310105.0,Changning,200050,31.22,121.42
310106.0,Jing'an,200040,31.23,121.45
310107.0,Putuo,200333,31.25,121.40
310109.0,Hongkou,200080,31.27,121.50


Drop the first row as it is not a district

In [176]:
df.drop(df.loc[df['Division']=='Shanghai'].index, inplace=True)
df

,Division,Postal code,Latitude,Longitude
Division code[104],,,,
310101.0,Huangpu,200001,31.23,121.48
NaN,Luwan,200020,31.22,121.47
NaN,Zhabei,200070,31.25,121.45
310104.0,Xuhui,200030,31.18,121.43
310105.0,Changning,200050,31.22,121.42
310106.0,Jing'an,200040,31.23,121.45
310107.0,Putuo,200333,31.25,121.40
310109.0,Hongkou,200080,31.27,121.50
310110.0,Yangpu,200082,31.27,121.52


Rename columns so that they make more sense

In [177]:
df.rename(columns={'Division code[104]': 'Division Code', 'Division': 'District'}, inplace=True)
df

,District,Postal code,Latitude,Longitude
Division code[104],,,,
310101.0,Huangpu,200001,31.23,121.48
NaN,Luwan,200020,31.22,121.47
NaN,Zhabei,200070,31.25,121.45
310104.0,Xuhui,200030,31.18,121.43
310105.0,Changning,200050,31.22,121.42
310106.0,Jing'an,200040,31.23,121.45
310107.0,Putuo,200333,31.25,121.40
310109.0,Hongkou,200080,31.27,121.50
310110.0,Yangpu,200082,31.27,121.52


Finalise the dataset which will be worked with

In [178]:
df.reset_index(drop=True, inplace=True)
df

,District,Postal code,Latitude,Longitude
0,Huangpu,200001,31.23,121.48
1,Luwan,200020,31.22,121.47
2,Zhabei,200070,31.25,121.45
3,Xuhui,200030,31.18,121.43
4,Changning,200050,31.22,121.42
5,Jing'an,200040,31.23,121.45
6,Putuo,200333,31.25,121.40
7,Hongkou,200080,31.27,121.50
8,Yangpu,200082,31.27,121.52
9,Minhang,201100,31.12,121.38


In [179]:
latitude_shanghai = 31.23
longitude_shanghai = 121.47

### Map of Shanghai

In [180]:
map_shanghai = folium.Map(location=[latitude_shanghai, longitude_shanghai], zoom_start=10)

for lat, lng, dis in zip(df['Latitude'], df['Longitude'], df['District']):
    
    label = '{}'.format(dis)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_shanghai)  
    
map_shanghai

In [181]:
CLIENT_ID = 'SNOQG0WPISU4BZZWSZJGJJDYN14XDF1JSR1A044BIA31R5X4' # your Foursquare ID
CLIENT_SECRET = 'KFJRX2M1TJ5VBK2BCHKPTKO0TJTMCBDOHDIZ1ZPUH501PGLF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: SNOQG0WPISU4BZZWSZJGJJDYN14XDF1JSR1A044BIA31R5X4
CLIENT_SECRET:KFJRX2M1TJ5VBK2BCHKPTKO0TJTMCBDOHDIZ1ZPUH501PGLF


In [182]:
district_latitude = df.loc[0, 'Latitude'] # district latitude value
district_longitude = df.loc[0, 'Longitude'] # district longitude value

district_name = df.loc[0, 'District'] # district name

print('Latitude and longitude values of {} are {}, {}.'.format(district_name, 
                                                               district_latitude, 
                                                               district_longitude))

Latitude and longitude values of Huangpu are 31.23, 121.48.


In [183]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    district_latitude, 
    district_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SNOQG0WPISU4BZZWSZJGJJDYN14XDF1JSR1A044BIA31R5X4&client_secret=KFJRX2M1TJ5VBK2BCHKPTKO0TJTMCBDOHDIZ1ZPUH501PGLF&v=20180605&ll=31.23,121.48&radius=500&limit=100'

In [184]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60382c947d0121680c1a8497'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Běixīn qiáo',
  'headerFullLocation': 'Běixīn qiáo, Shanghai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 49,
  'suggestedBounds': {'ne': {'lat': 31.234500004500006,
    'lng': 121.48525276546262},
   'sw': {'lat': 31.225499995499995, 'lng': 121.47474723453739}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5801b06138facdba59353259',
       'name': 'Campanile Hotel and Restaurant',
       'location': {'address': '33 Fujian S Rd',
        'lat': 31.232123106104904,
        'lng': 121.47914381462995,
        'labeledLatLngs': [{'label': 'display',


In [185]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [186]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Campanile Hotel and Restaurant,Hotel,31.232123,121.479144
1,Yu Garden Café,Café,31.229482,121.483501
2,台北纯K,Karaoke Bar,31.230826,121.477331
3,Club Lounge,Hotel Bar,31.229391,121.483472
4,Renaissance Shanghai Yu Garden Hotel (上海豫园万丽酒店),Hotel,31.229458,121.483456


In [187]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


In [188]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [189]:
shanghai_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Huangpu
Luwan
Zhabei
Xuhui
Changning
Jing'an
Putuo
Hongkou
Yangpu
Minhang
Baoshan
Jiading
Pudong
Jinshan
Songjiang
Qingpu
Fengxian
Chongming


In [190]:
print('There are {} Uniques Categories.'.format(len(shanghai_venues['Venue Category'].unique())))
shanghai_venues.groupby('District').count()

There are 114 Uniques Categories.


,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Baoshan,10,10,10,10,10,10
Changning,58,58,58,58,58,58
Chongming,4,4,4,4,4,4
Fengxian,4,4,4,4,4,4
Hongkou,7,7,7,7,7,7
Huangpu,37,37,37,37,37,37
Jiading,4,4,4,4,4,4
Jing'an,100,100,100,100,100,100
Jinshan,5,5,5,5,5,5


In [191]:
# one hot encoding
shanghai_onehot = pd.get_dummies(shanghai_venues[['Venue Category']], prefix="", prefix_sep="")

# add district column back to dataframe
shanghai_onehot['District'] = shanghai_venues['District'] 

# move district column to the first column
fixed_columns = [shanghai_onehot.columns[-1]] + list(shanghai_onehot.columns[:-1])
shanghai_onehot = shanghai_onehot[fixed_columns]
shanghai_grouped = shanghai_onehot.groupby('District').mean().reset_index()
shanghai_onehot

,District,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,Boutique,Brazilian Restaurant,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Convention Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Flower Shop,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,Hong Kong Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lingerie Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Mobile Phone Shop,Multiplex,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Photography Studio,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Squash Court,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant
0,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Huangpu,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Huangpu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [192]:
num_top_venues = 5
for hood in shanghai_grouped['District']:
    print("---- "+hood+" ----")
    temp =shanghai_grouped[shanghai_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Baoshan ----
                  venue  freq
0         Shopping Mall   0.3
1               Stadium   0.1
2         Metro Station   0.1
3           Coffee Shop   0.1
4  Fast Food Restaurant   0.1


---- Changning ----
                 venue  freq
0   Chinese Restaurant  0.16
1  Japanese Restaurant  0.12
2          Coffee Shop  0.10
3             Beer Bar  0.03
4       Clothing Store  0.03


---- Chongming ----
                     venue  freq
0       Chinese Restaurant  0.50
1                     Park  0.25
2                    Beach  0.25
3      American Restaurant  0.00
4  New American Restaurant  0.00


---- Fengxian ----
                 venue  freq
0     Asian Restaurant  0.25
1          Auto Garage  0.25
2          Pizza Place  0.25
3          Bus Station  0.25
4  American Restaurant  0.00


---- Hongkou ----
                venue  freq
0               Plaza  0.14
1       Shopping Mall  0.14
2  Chinese Restaurant  0.14
3                Park  0.14
4           Multiplex  0.14


-

In [193]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [194]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = shanghai_grouped['District']

for ind in np.arange(shanghai_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(shanghai_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baoshan,Shopping Mall,Coffee Shop,Metro Station,Department Store,Chinese Restaurant,Office,Fast Food Restaurant,Stadium,Gym / Fitness Center,Electronics Store
1,Changning,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Noodle House,Beer Bar,Café,Clothing Store,Lounge,Pizza Place
2,Chongming,Chinese Restaurant,Park,Beach,Yunnan Restaurant,Frozen Yogurt Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant
3,Fengxian,Pizza Place,Asian Restaurant,Auto Garage,Bus Station,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
4,Hongkou,Plaza,Park,Chinese Restaurant,Shopping Mall,New American Restaurant,Music Venue,Multiplex,Fast Food Restaurant,Deli / Bodega,Department Store


### K-means clustering

In [195]:
district_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baoshan,Shopping Mall,Coffee Shop,Metro Station,Department Store,Chinese Restaurant,Office,Fast Food Restaurant,Stadium,Gym / Fitness Center,Electronics Store
1,Changning,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Noodle House,Beer Bar,Café,Clothing Store,Lounge,Pizza Place
2,Chongming,Chinese Restaurant,Park,Beach,Yunnan Restaurant,Frozen Yogurt Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant
3,Fengxian,Pizza Place,Asian Restaurant,Auto Garage,Bus Station,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
4,Hongkou,Plaza,Park,Chinese Restaurant,Shopping Mall,New American Restaurant,Music Venue,Multiplex,Fast Food Restaurant,Deli / Bodega,Department Store
5,Huangpu,Hotel,Chinese Restaurant,Bookstore,Coffee Shop,Noodle House,Concert Hall,Seafood Restaurant,Sandwich Place,Restaurant,Peking Duck Restaurant
6,Jiading,Stadium,Convenience Store,Chinese Restaurant,Bus Stop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant
7,Jing'an,Coffee Shop,Hotel,Japanese Restaurant,Bakery,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Theater,Hotel Bar,Shanghai Restaurant
8,Jinshan,Italian Restaurant,Shopping Mall,Convenience Store,Chinese Restaurant,Coffee Shop,Athletics & Sports,Auto Garage,Department Store,Dessert Shop,Dim Sum Restaurant
9,Luwan,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Park,Hotel,Shopping Mall,Restaurant,Vegetarian / Vegan Restaurant,Nightclub


In [196]:
shanghai_grouped_clustering = shanghai_grouped.drop('District', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(shanghai_grouped_clustering)
kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [197]:
district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

shanghai_merged =df.iloc[:18,:]

# merge shanghai_grouped with shanghai_data to add latitude/longitude for each neighborhood
shanghai_merged = shanghai_merged.join(district_venues_sorted.set_index('District'), on='District')

shanghai_merged.head() # check the last columns!

,District,Postal code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Huangpu,200001,31.23,121.48,0,Hotel,Chinese Restaurant,Bookstore,Coffee Shop,Noodle House,Concert Hall,Seafood Restaurant,Sandwich Place,Restaurant,Peking Duck Restaurant
1,Luwan,200020,31.22,121.47,0,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Park,Hotel,Shopping Mall,Restaurant,Vegetarian / Vegan Restaurant,Nightclub
2,Zhabei,200070,31.25,121.45,0,Coffee Shop,Fast Food Restaurant,Hotel,Art Gallery,Chinese Restaurant,Café,American Restaurant,Lounge,New American Restaurant,Convenience Store
3,Xuhui,200030,31.18,121.43,0,Fast Food Restaurant,Coffee Shop,Metro Station,Hotel,Chinese Restaurant,Bus Station,Brewery,Mobile Phone Shop,Scandinavian Restaurant,Squash Court
4,Changning,200050,31.22,121.42,0,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Noodle House,Beer Bar,Café,Clothing Store,Lounge,Pizza Place


In [198]:
shanghai_merged

,District,Postal code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Huangpu,200001,31.23,121.48,0,Hotel,Chinese Restaurant,Bookstore,Coffee Shop,Noodle House,Concert Hall,Seafood Restaurant,Sandwich Place,Restaurant,Peking Duck Restaurant
1,Luwan,200020,31.22,121.47,0,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Park,Hotel,Shopping Mall,Restaurant,Vegetarian / Vegan Restaurant,Nightclub
2,Zhabei,200070,31.25,121.45,0,Coffee Shop,Fast Food Restaurant,Hotel,Art Gallery,Chinese Restaurant,Café,American Restaurant,Lounge,New American Restaurant,Convenience Store
3,Xuhui,200030,31.18,121.43,0,Fast Food Restaurant,Coffee Shop,Metro Station,Hotel,Chinese Restaurant,Bus Station,Brewery,Mobile Phone Shop,Scandinavian Restaurant,Squash Court
4,Changning,200050,31.22,121.42,0,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Noodle House,Beer Bar,Café,Clothing Store,Lounge,Pizza Place
5,Jing'an,200040,31.23,121.45,0,Coffee Shop,Hotel,Japanese Restaurant,Bakery,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Theater,Hotel Bar,Shanghai Restaurant
6,Putuo,200333,31.25,121.40,1,Seafood Restaurant,Fish Market,Metro Station,Yunnan Restaurant,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
7,Hongkou,200080,31.27,121.50,0,Plaza,Park,Chinese Restaurant,Shopping Mall,New American Restaurant,Music Venue,Multiplex,Fast Food Restaurant,Deli / Bodega,Department Store
8,Yangpu,200082,31.27,121.52,0,Hotel,Park,Chinese Restaurant,Metro Station,Flower Shop,Yunnan Restaurant,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant
9,Minhang,201100,31.12,121.38,0,Bakery,Café,Fast Food Restaurant,Plaza,Yunnan Restaurant,French Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


In [199]:
kclusters = 10

In [200]:
# create map
map_clusters = folium.Map(location=[latitude_shanghai, longitude_shanghai], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, dis , cluster in zip(shanghai_merged['Latitude'], 
                                   shanghai_merged['Longitude'], 
                                   shanghai_merged['District'], 
                                   shanghai_merged['Cluster Labels']):
    label = folium.Popup(str(dis) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']


### Examine clusters

In [201]:
shanghai_merged.loc[shanghai_merged['Cluster Labels'] == 0, shanghai_merged.columns[[0] + list(range(5, shanghai_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Huangpu,Hotel,Chinese Restaurant,Bookstore,Coffee Shop,Noodle House,Concert Hall,Seafood Restaurant,Sandwich Place,Restaurant,Peking Duck Restaurant
1,Luwan,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Park,Hotel,Shopping Mall,Restaurant,Vegetarian / Vegan Restaurant,Nightclub
2,Zhabei,Coffee Shop,Fast Food Restaurant,Hotel,Art Gallery,Chinese Restaurant,Café,American Restaurant,Lounge,New American Restaurant,Convenience Store
3,Xuhui,Fast Food Restaurant,Coffee Shop,Metro Station,Hotel,Chinese Restaurant,Bus Station,Brewery,Mobile Phone Shop,Scandinavian Restaurant,Squash Court
4,Changning,Chinese Restaurant,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Noodle House,Beer Bar,Café,Clothing Store,Lounge,Pizza Place
5,Jing'an,Coffee Shop,Hotel,Japanese Restaurant,Bakery,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Theater,Hotel Bar,Shanghai Restaurant
7,Hongkou,Plaza,Park,Chinese Restaurant,Shopping Mall,New American Restaurant,Music Venue,Multiplex,Fast Food Restaurant,Deli / Bodega,Department Store
8,Yangpu,Hotel,Park,Chinese Restaurant,Metro Station,Flower Shop,Yunnan Restaurant,French Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant
9,Minhang,Bakery,Café,Fast Food Restaurant,Plaza,Yunnan Restaurant,French Restaurant,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
10,Baoshan,Shopping Mall,Coffee Shop,Metro Station,Department Store,Chinese Restaurant,Office,Fast Food Restaurant,Stadium,Gym / Fitness Center,Electronics Store


In [202]:
shanghai_merged.loc[shanghai_merged['Cluster Labels'] == 1, shanghai_merged.columns[[0] + list(range(5, shanghai_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Putuo,Seafood Restaurant,Fish Market,Metro Station,Yunnan Restaurant,Frozen Yogurt Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [203]:
shanghai_merged.loc[shanghai_merged['Cluster Labels'] == 2, shanghai_merged.columns[[0] + list(range(5, shanghai_merged.shape[1]))]]

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chongming,Chinese Restaurant,Park,Beach,Yunnan Restaurant,Frozen Yogurt Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant


So in the end there are three clusters, where one consists of only the district Putuo, another of only the island district Chongming, while the other cluster contains the rest of the districts. Putuo is one of the smaller districts and is not in the center, but has the same population as some of the districts in the center. Chongming is more of an island that is a little further from Shanghai and has more nature than the big city. This may show that there are quite some things to do here that may not be in other parts, so tourists may get some more value by traveling to this district. One thing to note is that the software we are using is based on user input. Most of these people are from outside China, meaning that many places are probably not mapped. This may have influenced the results quite heavily.